In [1]:
import ROOT
import numpy as np
import copy
import os

Welcome to JupyROOT 6.22/06


In [2]:
# Define momentum ranges
momentum_ranges = np.arange(0,10,1)
num_ranges = len(momentum_ranges)

In [3]:
# Define plot marker styles and colors
marker_styles = [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
marker_colors = [1]

In [11]:
def plot_contamination(project_dir, channel, version, reco_info, true_info, momentum_ranges):
    if(not os.path.exists(f"{project_dir}/systematics/false_id")):
        os.makedirs(f"{project_dir}/systematics/false_id")
    if(not os.path.exists(f"{project_dir}/systematics/false_id/{channel}")):
        os.makedirs(f"{project_dir}/systematics/false_id/{channel}")
    
    # Open input file and get TTree
    if(version=="MC_RGA_inbending"):
        suffix="MC_RGA_30*.root"
    elif(version=="MC_RGA_outbending"):
        suffix="MC_RGA_3313_*.root"
    
    tree = ROOT.TChain("EventTree")
    tree.Add(f"{project_dir}/volatile/data/{channel}/{suffix}")
    CHANNEL=channel.replace("piplus","#pi^{+}").replace("piminus","#pi^{-}").replace("pi0","#pi^{0}").replace("_","")
    
    reco_pid = reco_info[0]
    reco_name = reco_info[1]
    true_pid = true_info[0]
    true_name = true_info[1]
    true_cut = (f"&&truepid=={true_info[0]}" if true_info[0] else "")
    true_title = (f"({true_name})" if true_info[1] else "")
    # Define lists to store TH1F objects
    hist_list = []
    # Create TCanvas and TLegend
    canvas = ROOT.TCanvas("canvas", "Electron Contamination vs. Theta",800,600)
    legend = ROOT.TLegend(0.6, 0.6, 0.9, 0.9)
    ROOT.gStyle.SetOptStat(0)
    # Loop over momentum ranges and create TH1F for each range
    for i in range(len(momentum_ranges)-1):
        pmin=momentum_ranges[i]
        pmax=momentum_ranges[i+1]
        # Create TH1F for this momentum range
        hist_false = ROOT.TH1F("hist_false{}".format(i), "{}{} Contamination vs. Theta ( {} dihadrons)".format(reco_name,true_title,CHANNEL), 15, 8, 35)
        hist_true = ROOT.TH1F("hist_true{}".format(i), "{}{} Contamination vs. Theta ( {} dihadrons)".format(reco_name,true_title,CHANNEL), 15, 8, 35)
        # Fill histograms with contamination percentages
        tree.Draw("theta*180/3.1415 >> hist_false{}".format(i), "pid == {} && truepid != {} {} && truepid != -999  && p > {} && p < {}".format(reco_pid,reco_pid,true_cut,pmin, pmax), "goff")
        tree.Draw("theta*180/3.1415 >> hist_true{}".format(i), "pid == {}  && p > {} && p < {}".format(reco_pid,pmin, pmax), "goff")
        # Divide histograms to get contamination percentage
        hist_false.Divide(hist_true)
        # Add TH1F to list
        hist_list.append(hist_false)
        # Set histogram style and add to legend
        hist_list[-1].SetMarkerColor(marker_colors[i%len(marker_colors)])
        hist_list[-1].SetMarkerStyle(marker_styles[i//len(marker_colors)])
        hist_list[-1].SetMarkerSize(1.5)
        legend.AddEntry(hist_list[-1], "{:.1f} < p < {:.1f} GeV".format(pmin, pmax), "p")
        # Draw TH1F on canvas
        hist_list[-1].Draw("P same" if i!=0 else "P")
    # Set canvas properties and draw legend
    hist_list[0].GetYaxis().SetRangeUser(0.00001,1)
    canvas.SetLogy(1)
    canvas.SetGrid()
    legend.Draw("same")
    # Set axis labels and draw on canvas
    hist_list[0].GetXaxis().SetTitle("Theta [deg]")
    hist_list[0].GetYaxis().SetTitle("Contamination (%)")
    canvas.SaveAs(f"{project_dir}/systematics/false_id/{channel}/figure_{reco_pid}_as_{true_pid}_{version}.png")

In [12]:
project_dir="/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0"
versions = ["MC_RGA_inbending","MC_RGA_outbending"]
channels = ["piplus_piminus", "piplus_pi0", "piplus_piplus","pi0_pi0","piminus_pi0", "piminus_piminus"]

In [13]:
def get_ri_ti(channel):
    reco_info=[[11,"e^{-}"]]
    true_info=[[-211,"#pi^{-}"]]
    if("piplus" in channel):
        reco_info.append([211,"#pi^{+}"])
        true_info.append([321,"K^{+}"])
    if("piminus" in channel):
        reco_info.append([-211,"#pi^{-}"])
        true_info.append([-321,"K^{-}"])
    if("pi0" in channel):
        reco_info.append([22,"#gamma"])
        true_info.append([2112,"n"])
    return reco_info,true_info

In [14]:
for version in versions:
    for channel in channels:
        reco_info,true_info=get_ri_ti(channel)
        for ri,ti in zip(reco_info,true_info):
            print(version,channel,"\t",ri[0],"as",ti[0])
            plot_contamination(project_dir,channel,version,ri,ti,momentum_ranges)

MC_RGA_inbending piplus_piminus 	 11 as -211
MC_RGA_inbending piplus_piminus 	 211 as 321
MC_RGA_inbending piplus_piminus 	 -211 as -321
MC_RGA_inbending piplus_pi0 	 11 as -211
MC_RGA_inbending piplus_pi0 	 211 as 321
MC_RGA_inbending piplus_pi0 	 22 as 2112
MC_RGA_inbending piplus_piplus 	 11 as -211
MC_RGA_inbending piplus_piplus 	 211 as 321
MC_RGA_inbending pi0_pi0 	 11 as -211
MC_RGA_inbending pi0_pi0 	 22 as 2112
MC_RGA_inbending piminus_pi0 	 11 as -211
MC_RGA_inbending piminus_pi0 	 -211 as -321
MC_RGA_inbending piminus_pi0 	 22 as 2112
MC_RGA_inbending piminus_piminus 	 11 as -211
MC_RGA_inbending piminus_piminus 	 -211 as -321
MC_RGA_outbending piplus_piminus 	 11 as -211
MC_RGA_outbending piplus_piminus 	 211 as 321
MC_RGA_outbending piplus_piminus 	 -211 as -321
MC_RGA_outbending piplus_pi0 	 11 as -211
MC_RGA_outbending piplus_pi0 	 211 as 321
MC_RGA_outbending piplus_pi0 	 22 as 2112
MC_RGA_outbending piplus_piplus 	 11 as -211
MC_RGA_outbending piplus_piplus 	 211 as 32

Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/false_id/piplus_piminus/figure_11_as_-211_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/false_id/piplus_piminus/figure_211_as_321_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/false_id/piplus_piminus/figure_-211_as_-321_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/false_id/piplus_pi0/figure_11_as_-211_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/false_id/piplus_pi0/figure_211_as_321_MC_RGA_inbending.png has been created
Info in <TCanvas::Print>: png file /work/clas12/use

In [16]:
import os

# Define the output filename for the LaTeX file
latex_filename = f"{project_dir}/systematics/false_id/plots.tex"

# Open the LaTeX file for writing
with open(latex_filename, "w") as f:
    # Write the LaTeX document preamble
    f.write("\\documentclass{article}\n")
    f.write("\\usepackage{graphicx}\n")
    #f.write("\\usepackage{caption}\n")
    f.write("\\usepackage{geometry}\n")
    f.write("\\usepackage{fancyhdr}\n")

    f.write("\\geometry{margin=2cm}\n") # Set the margin to 1 cm
    f.write("\\usepackage[font=large]{caption}\n") # Set the caption font size to large
    f.write("\\begin{document}\n")
    f.write("\\pagestyle{fancy}\n")
    f.write("\\fancyhf{}\n")
    f.write("\\fancyhead[L]{\\leftmark\\,\\,\\,\\,\\rightmark}\n")
    f.write("\\fancyhead[R]{\\thepage}\n")
    # Add a table of contents
    f.write("\\tableofcontents\n")
    # Loop over all channels
    for channel in channels:
        CHANNEL=channel.replace("piplus","$\\pi^{+}$").replace("piminus","$\\pi^{-}$").replace("pi0","$\\pi^{0}$").replace("_","")
        f.write(f"\\section{{{CHANNEL} dihadrons}}") # Add a section for the channel
        
        # Loop over all versions
        for version in versions:
            VERSION=version.replace("_"," ")
            f.write(f"\\subsection{{{VERSION}}}") # Add a subsection for the version
            reco_info,true_info=get_ri_ti(channel)
            # Loop over all binstructs
            for ri,ti in zip(reco_info,true_info):
                # Define the plot filename and caption
                plotname = f'{project_dir}/systematics/false_id/{channel}/figure_{ri[0]}_as_{ti[0]}_{version}.png'
                caption = f"NEWLINE{CHANNEL}NEWLINEMonte Carlo Version={version}"
                #plotname = plotname.replace("_","\_")
                caption = caption.replace("_","\_").replace("NEWLINE","\\\\")
                # Write the LaTeX code for the plot and caption
                f.write("\\begin{figure}[ht]\n")
                f.write(f"\\includegraphics[width=\\textwidth]{{{plotname}}}\n")
                f.write(f"\\captionsetup{{font=large}}\n") # Set the caption font size to large
                f.write(f"\\caption{{{caption}}}\n")
                f.write("\\end{figure}\n")
            f.write("\\clearpage")
    # Write the LaTeX document postamble
    f.write("\\end{document}\n")
    
# Get the directory part of the LaTeX filename
latex_dirname = os.path.dirname(latex_filename)

# Save the current working directory
cwd = os.getcwd()

# Change the current working directory to the LaTeX directory
os.chdir(latex_dirname)

# Compile the LaTeX file into a PDF
os.system(f"pdflatex {latex_filename}")
os.system(f"pdflatex {latex_filename}")

# Change the current working directory back to the original
os.chdir(cwd)

This is pdfTeX, Version 3.14159265-2.6-1.40.20 (TeX Live 2019/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode

(/work/clas12/users/gmat/clas12/clas12_dihadrons/projects/ana_v0/systematics/fa
lse_id/plots.tex
LaTeX2e <2020-02-02> patch level 2
L3 programming layer <2020-02-14>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/trig.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-cfg/graphics.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-def/pdftex.def)))
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/generic/ifte